## Step 1: Installing Meridian

In [5]:
!pip install --upgrade google-meridian

  Using cached arviz-0.22.0-py3-none-any.whl.metadata (8.9 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached immutabledict-4.2.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached natsort-7.1.1-py3-none-any.whl.metadata (22 kB)
  Using cached numpy-2.3.2-cp312-cp312-macosx_14_0_x86_64.whl.metadata (62 kB)
  Using cached pandas-2.3.1-cp312-cp312-macosx_10_13_x86_64.whl.metadata (91 kB)
  Using cached scipy-1.16.1-cp312-cp312-macosx_14_0_x86_64.whl.metadata (61 kB)
INFO: pip is looking at multiple versions of google-meridian to determine which version is compatible with other requirements. This could take a while.
  Using cached google_meridian-1.1.4-py3-none-any.whl.metadata (22 kB)
  Using cached google_meridian-1.1.3-py3-none-any.whl.metadata (22 kB)
  Using cached google_meridian-1.1.2-py3-none-any.whl.metadata (22 kB)
  Using cached google_meridian-1.1.1-py3-none-any.whl.metadata (22 kB)
  U

In [1]:
import arviz as az
import IPython
from meridian import constants
from meridian.analysis import analyzer
from meridian.analysis import formatter
from meridian.analysis import optimizer
from meridian.analysis import summarizer
from meridian.analysis import visualizer
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import prior_distribution
from meridian.model import spec
import numpy as np
import pandas as pd
# check if GPU is available
from psutil import virtual_memory
import tensorflow as tf
import tensorflow_probability as tfp

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print(
    'Num GPUs Available: ',
    len(tf.config.experimental.list_physical_devices('GPU')),
)
print(
    'Num CPUs Available: ',
    len(tf.config.experimental.list_physical_devices('CPU')),
)

2025-08-02 13:01:57.134900: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Your runtime has 17.2 gigabytes of available RAM

Num GPUs Available:  0
Num CPUs Available:  1


## Step 2: Loading the data

In [11]:
import os
os.chdir('/Users/Office/Desktop/Projects/Meridian-Demo/')

In [12]:
import os
print(os.getcwd())

/Users/Office/Desktop/Projects/Meridian-Demo


In [13]:
df = pd.read_csv('./data/demo.csv')

Create a `CoordToColumns` object specifying which columns represent controls, KPI, media, media spend, reach, frequency, and RF spend. And then set up dictionaries that map each media-related column to its corresponding channel. These mappings are used to initialize a `CsvDataLoader`, which loads and processes the data from the CSV file for use in the modeling workflow.

In [14]:
channels = ["Channel0", "Channel1", "Channel2"]

coord_to_columns = load.CoordToColumns(
    controls=[
        "sentiment_score_control",
        "competitor_activity_score_control"
    ],
    kpi='conversions',
    media=[f"{channel}_impression" for channel in channels],
    media_spend=[f"{channel}_spend" for channel in channels],
    reach=["Channel3_reach"],
    frequency=["Channel3_frequency"],
    rf_spend=["Channel3_spend"],
)

correct_media_to_channel = {
    f"{channel}_impression": channel for channel in channels 
}

correct_media_spend_to_channel = {
    f"{channel}_spend": channel for channel in channels
}

correct_reach_to_channel = {
    "Channel3_reach": "Channel3"
}

correct_frequency_to_channel = {
    "Channel3_frequency": "Channel3"
}

correct_rf_spend_to_channel = {
    "Channel3_spend": "Channel3"
}

loader = load.CsvDataLoader(
    csv_path="./data/demo.csv",
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
    reach_to_channel=correct_reach_to_channel,
    frequency_to_channel=correct_frequency_to_channel,
    rf_spend_to_channel=correct_rf_spend_to_channel,
)

data = loader.load()

/Users/Office/Desktop/Projects/Meridian-Demo/.venv/lib/python3.12/site-packages/meridian/data/load.py:1020: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df[geo_column_name] = self.df[geo_column_name].replace(
/Users/Office/Desktop/Projects/Meridian-Demo/.venv/lib/python3.12/site-packages/meridian/data/load.py:201: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if (constants.GEO) not in self.dataset.dims.keys():
/Users/Office/Desktop/Projects/Meridian-Demo/.venv/lib/python3.12/site-packages/meridian/data/load.py:231: FutureWarning: The return type of `Data

## Step 3: Configuring the model

Inititalize the `Meridian` class by passing the loaded data and the customized model specification.

In [15]:
roi_rf_mu = 0.2  # Mu for ROI prior for each RF channel.
roi_rf_sigma = 0.9  # Sigma for ROI prior for each RF channel.
prior = prior_distribution.PriorDistribution(
    roi_rf=tfp.distributions.LogNormal(
        roi_rf_mu, roi_rf_sigma, name=constants.ROI_RF
    )
)
model_spec = spec.ModelSpec(prior=prior)

mmm = model.Meridian(input_data=data, model_spec=model_spec)

I0000 00:00:1754132766.292109 4804458 service.cc:145] XLA service 0x7f7924ba9570 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754132766.293615 4804458 service.cc:153]   StreamExecutor device (0): Host, Default Version
2025-08-02 13:06:06.295798: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1754132766.590276 4804458 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Utilize the `sample_prior()` and `sample_posterior()` methods to generate samples from the prior and posterior distributions of the model parameters.

In [16]:
%%time
mmm.sample_prior(500)
mmm.sample_posterior(
    n_chains=4, n_adapt=500, n_burnin=500, n_keep=1000, seed=1
)

/Users/Office/Desktop/Projects/Meridian-Demo/.venv/lib/python3.12/site-packages/meridian/model/prior_distribution.py:747: UserWarning: Consider setting custom ROI priors, as kpi_type was specified as `non_revenue` with no `revenue_per_kpi` being set. Otherwise, the total media contribution prior will be used with `p_mean=0.4` and `p_sd=0.2`. Further documentation available at  https://developers.google.com/meridian/docs/advanced-modeling/unknown-revenue-kpi-custom#set-total-paid-media-contribution-prior
  warnings.warn(
/Users/Office/Desktop/Projects/Meridian-Demo/.venv/lib/python3.12/site-packages/meridian/model/prior_distribution.py:767: UserWarning: Consider setting custom ROI priors, as kpi_type was specified as `non_revenue` with no `revenue_per_kpi` being set. Otherwise, the total media contribution prior will be used with `p_mean=0.4` and `p_sd=0.2`. Further documentation available at  https://developers.google.com/meridian/docs/advanced-modeling/unknown-revenue-kpi-custom#set-t

CPU times: user 1h 39min 32s, sys: 22min 19s, total: 2h 1min 51s
Wall time: 1h 12min 52s


## Step 4: Model diagnosis

Once the model is built, next step is to access the convergence, the following generates r-hat statistics, values of r-hat closer to 1 means convergence

In [18]:
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.plot_rhat_boxplot()

alt.LayerChart(...)

Plot the ROI posterior distribution against the ROI prior distribution for each media channel as follows

In [19]:
model_diagnostics.plot_prior_and_posterior_distribution()


alt.FacetChart(...)

The following lets you compare expected sales with the actual sales

In [20]:
model_fit = visualizer.ModelFit(mmm)
model_fit.plot_model_fit()

alt.LayerChart(...)

## Step 5: Model results

To generate the two-page HTML summary output, first initialize the `Summarizer` class with the model object. Then, use the `output_model_results_summary` method, providing the filename, file path, start date, and end date to generate and save the summary to your specified location.

In [21]:
mmm_summarizer = summarizer.Summarizer(mmm)

In [22]:
filepath = './reports/'
start_date = '2021-01-25'
end_date = '2024-01-15'
mmm_summarizer.output_model_results_summary(
    'summary_output.html', filepath, start_date, end_date
)

/Users/Office/Desktop/Projects/Meridian-Demo/.venv/lib/python3.12/site-packages/meridian/analysis/analyzer.py:261: UserWarning: A `media_spend` value was passed in the `new_data` argument. This is not supported and will be ignored.
  warnings.warn(
/Users/Office/Desktop/Projects/Meridian-Demo/.venv/lib/python3.12/site-packages/meridian/analysis/analyzer.py:261: UserWarning: A `rf_spend` value was passed in the `new_data` argument. This is not supported and will be ignored.
  warnings.warn(
/Users/Office/Desktop/Projects/Meridian-Demo/.venv/lib/python3.12/site-packages/meridian/analysis/analyzer.py:261: UserWarning: A `reach` value was passed in the `new_data` argument. This is not supported and will be ignored.
  warnings.warn(
/Users/Office/Desktop/Projects/Meridian-Demo/.venv/lib/python3.12/site-packages/meridian/analysis/analyzer.py:261: UserWarning: A `frequency` value was passed in the `new_data` argument. This is not supported and will be ignored.
  warnings.warn(
/Users/Office/D

The `MediaSummary` class is used to generate model results summaries. By default, it produces summary statistics using a 90% credible interval over the entire modeling period.

In [ ]:
media_summary = visualizer.MediaSummary(mmm)
media_summary.summary_table()

## Step 6: Budget optimization and generating report

You can select different scenarios for budget allocation. By default, the library finds the optimal allocation across channels for a specified budget to maximize return on investment (ROI).

To run the default Fixed Budget Scenario and maximize ROI, simply create an instance of the `BudgetOptimizer` class and call its `optimize()` method without any additional configuration.

In [ ]:
%%time
budget_optimizer = optimizer.BudgetOptimizer(mmm)
optimization_results = budget_optimizer.optimize()

Export the HTML optimization report

In [ ]:
filepath = './reports/'
optimization_results.output_optimization_summary(
    'optimization_output.html', filepath
)

In [ ]:
IPython.display.HTML(filename='/content/drive/MyDrive/optimization_output.html')

## Step 7: Saving the model

Save the model as follows

In [17]:
file_path = './models/demo_mmm.pkl'
model.save_mmm(mmm, file_path)

To load the model, run the following

In [ ]:
mmm = model.load_mmm(file_path)